In [2]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [41]:
import re
import numpy as np
import sys
import os
import pandas as pd
import string
import nltk

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import *

In [42]:
read_file = pd.read_excel('(usingGoogle)final_translated.xlsx', engine='openpyxl')
# read_file.to_csv (r'Dataset_A.csv', index = None, header=True)

In [43]:
df=pd.DataFrame(read_file)

In [44]:
df

,Unnamed: 0,Unnamed: 0.1,qid:,qstr,candidates,DBRECORDID,TITLE,ABSTRACT,AUTHOR,SOURCE,...,PUBLPLACE,INSTITUTION,TITLE_,ABSTRACT_,TITLE_lan,ABSTRACT_lan,title_en,abstract_en,TITLE_lan code(memo),ABSTRACT_lan code(memo)
0,0,0,943,mbsr,M30928765,M30928765,['The beauty we love: MBSR teacher education g...,['The paper begins with a brief overview of th...,"['Santorelli, Saki F']",['Current opinion in psychology'],...,NaN,NaN,The beauty we love MBSR teacher education goin...,The paper begins with a brief overview of the ...,en,en,The beauty we love MBSR teacher education goin...,The paper begins with a brief overview of the ...,NaN,NaN
1,1,1,943,mbsr,M30928764,M30928764,['Building an ark: creating a vessel for the e...,['This paper is a response to an invitation by...,"['Santorelli, Saki F']",['Current opinion in psychology'],...,NaN,NaN,Building an ark creating a vessel for the educ...,This paper is a response to an invitation by t...,en,en,Building an ark creating a vessel for the educ...,This paper is a response to an invitation by t...,NaN,NaN
2,2,2,943,mbsr,M33155250,M33155250,['The use of online MBSR audio in medical staf...,"[""Objective!#!This study aims to survey medica...","['Liu, Y', 'Luo, S-X', 'Ye, J-L', 'Chen, Y-Z',...",['European review for medical and pharmacologi...,...,NaN,NaN,The use of online MBSR audio in medical staff ...,ObjectiveThis study aims to survey medical sta...,en,en,The use of online MBSR audio in medical staff ...,ObjectiveThis study aims to survey medical sta...,NaN,NaN
3,3,3,943,mbsr,M30137607,M30137607,['Translational genomic research: the role of ...,"[""Genetic variations of breast cancer survivor...","['Park, Jong Y', 'Lengacher, Cecile A', 'Reich...",['Translational behavioral medicine'],...,NaN,NaN,Translational genomic research the role of gen...,Genetic variations of breast cancer survivors ...,en,en,Translational genomic research the role of gen...,Genetic variations of breast cancer survivors ...,NaN,NaN
4,4,4,943,mbsr,M32848377,M32848377,['The Effect of Mindfulness-Based Stress Reduc...,['Background!#!Mindfulness-based programs have...,"['Marciniak, Rafał', 'Šumec, Rastislav', 'Vyhn...",['Clinical interventions in aging'],...,NaN,NaN,The Effect of MindfulnessBased Stress Reductio...,BackgroundMindfulnessbased programs have shown...,en,en,The Effect of MindfulnessBased Stress Reductio...,BackgroundMindfulnessbased programs have shown...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,9995,1172,low AND carb,M33499118,M33499118,['The Effect of Antioxidant and Anti-Inflammat...,"[""Chronic inflammation and increased oxidative...","['Katsimbri, Pelagia', 'Korakas, Emmanouil', '...","['Antioxidants (Basel, Switzerland)']",...,NaN,NaN,The Effect of Antioxidant and AntiInflammatory...,Chronic inflammation and increased oxidative s...,en,en,The Effect of Antioxidant and AntiInflammatory...,Chronic inflammation and increased oxidative s...,NaN,NaN
9996,9996,9996,1172,low AND carb,M32713393,M32713393,['Participant characteristics and self-reporte...,"[""Objective!#!To describe characteristics of s...","['Karlsen, Micaela C', 'Lichtenstein, Alice H'...",['Public health nutrition'],...,NaN,NaN,Participant characteristics and selfreported w...,ObjectiveTo describe characteristics of selfid...,en,en,Participant characteristics and selfreported w...,ObjectiveTo describe characteristics of selfid...,NaN,NaN
9997,9997,9997,1172,low AND carb,M27391484,M27391484,['Effect of glycemic index and carbohydrate in...,['Background!#!Replacing carbohydrate with pro...,"['Juraschek, Stephen P', 'Chang, Alex R', 'App...",['BMC nephrology'],...,NaN,NaN,Effect of glycemic index and carbohydrate inta...,BackgroundReplacing carbohydrate with protein ...,en,en,Effect of glycemic index and carbohydrate inta...,BackgroundReplacing carbohydrate with protein ...,NaN,NaN
9998,9998,9998,1172,low AND carb,M30614551,M30614551,['Hormonal and metabolic effects of carbohydra

In [45]:
df['Preprocessed_Abstract']=df['abstract_en'].fillna("")
df['Preprocessed_Title']=df['title_en'].fillna("")

In [14]:
df

,Unnamed: 0,Unnamed: 0.1,qid:,qstr,candidates,DBRECORDID,TITLE,ABSTRACT,AUTHOR,SOURCE,...,TITLE_,ABSTRACT_,TITLE_lan,ABSTRACT_lan,title_en,abstract_en,TITLE_lan code(memo),ABSTRACT_lan code(memo),Preprocessed_Abstract,Preprocessed_Title
0,0,0,943,mbsr,M30928765,M30928765,['The beauty we love: MBSR teacher education g...,['The paper begins with a brief overview of th...,"['Santorelli, Saki F']",['Current opinion in psychology'],...,The beauty we love MBSR teacher education goin...,The paper begins with a brief overview of the ...,en,en,The beauty we love MBSR teacher education goin...,The paper begins with a brief overview of the ...,NaN,NaN,The paper begins with a brief overview of the ...,The beauty we love MBSR teacher education goin...
1,1,1,943,mbsr,M30928764,M30928764,['Building an ark: creating a vessel for the e...,['This paper is a response to an invitation by...,"['Santorelli, Saki F']",['Current opinion in psychology'],...,Building an ark creating a vessel for the educ...,This paper is a response to an invitation by t...,en,en,Building an ark creating a vessel for the educ...,This paper is a response to an invitation by t...,NaN,NaN,This paper is a response to an invitation by t...,Building an ark creating a vessel for the educ...
2,2,2,943,mbsr,M33155250,M33155250,['The use of online MBSR audio in medical staf...,"[""Objective!#!This study aims to survey medica...","['Liu, Y', 'Luo, S-X', 'Ye, J-L', 'Chen, Y-Z',...",['European review for medical and pharmacologi...,...,The use of online MBSR audio in medical staff ...,ObjectiveThis study aims to survey medical sta...,en,en,The use of online MBSR audio in medical staff ...,ObjectiveThis study aims to survey medical sta...,NaN,NaN,ObjectiveThis study aims to survey medical sta...,The use of online MBSR audio in medical staff ...
3,3,3,943,mbsr,M30137607,M30137607,['Translational genomic research: the role of ...,"[""Genetic variations of breast cancer survivor...","['Park, Jong Y', 'Lengacher, Cecile A', 'Reich...",['Translational behavioral medicine'],...,Translational genomic research the role of gen...,Genetic variations of breast cancer survivors ...,en,en,Translational genomic research the role of gen...,Genetic variations of breast cancer survivors ...,NaN,NaN,Genetic variations of breast cancer survivors ...,Translational genomic research the role of gen...
4,4,4,943,mbsr,M32848377,M32848377,['The Effect of Mindfulness-Based Stress Reduc...,['Background!#!Mindfulness-based programs have...,"['Marciniak, Rafał', 'Šumec, Rastislav', 'Vyhn...",['Clinical interventions in aging'],...,The Effect of MindfulnessBased Stress Reductio...,BackgroundMindfulnessbased programs have shown...,en,en,The Effect of MindfulnessBased Stress Reductio...,BackgroundMindfulnessbased programs have shown...,NaN,NaN,BackgroundMindfulnessbased programs have shown...,The Effect of MindfulnessBased Stress Reductio...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,9995,1172,low AND carb,M33499118,M33499118,['The Effect of Antioxidant and Anti-Inflammat...,"[""Chronic inflammation and increased oxidative...","['Katsimbri, Pelagia', 'Korakas, Emmanouil', '...","['Antioxidants (Basel, Switzerland)']",...,The Effect of Antioxidant and AntiInflammatory...,Chronic inflammation and increased oxidative s...,en,en,The Effect of Antioxidant and AntiInflammatory...,Chronic inflammation and increased oxidative s...,NaN,NaN,Chronic inflammation and increased oxidative s...,The Effect of Antioxidant and AntiInflammatory...
9996,9996,9996,1172,low AND carb,M32713393,M32713393,['Participant characteristics and self-reporte...,"[""Objective!#!To describe characteristics of s...","['Karlsen, Micaela C', 'Lichtenstein, Alice H'...",['Public health nutrition'],...,Participant characteristics and selfreported w...,ObjectiveTo describe characteristics of selfid...,en,en,Participant characteristics and selfreported w...,ObjectiveTo describe characteristics of self

In [7]:
def list_to_string(text):
    abc=''.join(text)
   
    return abc

In [8]:
df['Preprocessing'] = df['ABSTRACT'].apply(list_to_string)

TypeError: can only join an iterable

In [15]:
df

,Unnamed: 0,Unnamed: 0.1,qid:,qstr,candidates,DBRECORDID,TITLE,ABSTRACT,AUTHOR,SOURCE,...,TITLE_,ABSTRACT_,TITLE_lan,ABSTRACT_lan,title_en,abstract_en,TITLE_lan code(memo),ABSTRACT_lan code(memo),Preprocessed_Abstract,Preprocessed_Title
0,0,0,943,mbsr,M30928765,M30928765,['The beauty we love: MBSR teacher education g...,['The paper begins with a brief overview of th...,"['Santorelli, Saki F']",['Current opinion in psychology'],...,The beauty we love MBSR teacher education goin...,The paper begins with a brief overview of the ...,en,en,The beauty we love MBSR teacher education goin...,The paper begins with a brief overview of the ...,NaN,NaN,The paper begins with a brief overview of the ...,The beauty we love MBSR teacher education goin...
1,1,1,943,mbsr,M30928764,M30928764,['Building an ark: creating a vessel for the e...,['This paper is a response to an invitation by...,"['Santorelli, Saki F']",['Current opinion in psychology'],...,Building an ark creating a vessel for the educ...,This paper is a response to an invitation by t...,en,en,Building an ark creating a vessel for the educ...,This paper is a response to an invitation by t...,NaN,NaN,This paper is a response to an invitation by t...,Building an ark creating a vessel for the educ...
2,2,2,943,mbsr,M33155250,M33155250,['The use of online MBSR audio in medical staf...,"[""Objective!#!This study aims to survey medica...","['Liu, Y', 'Luo, S-X', 'Ye, J-L', 'Chen, Y-Z',...",['European review for medical and pharmacologi...,...,The use of online MBSR audio in medical staff ...,ObjectiveThis study aims to survey medical sta...,en,en,The use of online MBSR audio in medical staff ...,ObjectiveThis study aims to survey medical sta...,NaN,NaN,ObjectiveThis study aims to survey medical sta...,The use of online MBSR audio in medical staff ...
3,3,3,943,mbsr,M30137607,M30137607,['Translational genomic research: the role of ...,"[""Genetic variations of breast cancer survivor...","['Park, Jong Y', 'Lengacher, Cecile A', 'Reich...",['Translational behavioral medicine'],...,Translational genomic research the role of gen...,Genetic variations of breast cancer survivors ...,en,en,Translational genomic research the role of gen...,Genetic variations of breast cancer survivors ...,NaN,NaN,Genetic variations of breast cancer survivors ...,Translational genomic research the role of gen...
4,4,4,943,mbsr,M32848377,M32848377,['The Effect of Mindfulness-Based Stress Reduc...,['Background!#!Mindfulness-based programs have...,"['Marciniak, Rafał', 'Šumec, Rastislav', 'Vyhn...",['Clinical interventions in aging'],...,The Effect of MindfulnessBased Stress Reductio...,BackgroundMindfulnessbased programs have shown...,en,en,The Effect of MindfulnessBased Stress Reductio...,BackgroundMindfulnessbased programs have shown...,NaN,NaN,BackgroundMindfulnessbased programs have shown...,The Effect of MindfulnessBased Stress Reductio...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,9995,1172,low AND carb,M33499118,M33499118,['The Effect of Antioxidant and Anti-Inflammat...,"[""Chronic inflammation and increased oxidative...","['Katsimbri, Pelagia', 'Korakas, Emmanouil', '...","['Antioxidants (Basel, Switzerland)']",...,The Effect of Antioxidant and AntiInflammatory...,Chronic inflammation and increased oxidative s...,en,en,The Effect of Antioxidant and AntiInflammatory...,Chronic inflammation and increased oxidative s...,NaN,NaN,Chronic inflammation and increased oxidative s...,The Effect of Antioxidant and AntiInflammatory...
9996,9996,9996,1172,low AND carb,M32713393,M32713393,['Participant characteristics and self-reporte...,"[""Objective!#!To describe characteristics of s...","['Karlsen, Micaela C', 'Lichtenstein, Alice H'...",['Public health nutrition'],...,Participant characteristics and selfreported w...,ObjectiveTo describe characteristics of selfid...,en,en,Participant characteristics and selfreported w...,ObjectiveTo describe characteristics of self

In [34]:
type(df['Preprocessed_Abstract'][4018])


int

In [29]:
df

,Unnamed: 0,Unnamed: 0.1,qid:,qstr,candidates,DBRECORDID,TITLE,ABSTRACT,AUTHOR,SOURCE,...,TITLE_,ABSTRACT_,TITLE_lan,ABSTRACT_lan,title_en,abstract_en,TITLE_lan code(memo),ABSTRACT_lan code(memo),Preprocessed_Abstract,Preprocessed_Title
0,0,0,943,mbsr,M30928765,M30928765,['The beauty we love: MBSR teacher education g...,"[[, ', T, h, e, , p, a, p, e, r, , b, e, g, ...","['Santorelli, Saki F']",['Current opinion in psychology'],...,The beauty we love MBSR teacher education goin...,The paper begins with a brief overview of the ...,en,en,The beauty we love MBSR teacher education goin...,The paper begins with a brief overview of the ...,NaN,NaN,The paper begins with a brief overview of the ...,The beauty we love MBSR teacher education goin...
1,1,1,943,mbsr,M30928764,M30928764,['Building an ark: creating a vessel for the e...,"[[, ', T, h, i, s, , p, a, p, e, r, , i, s, ...","['Santorelli, Saki F']",['Current opinion in psychology'],...,Building an ark creating a vessel for the educ...,This paper is a response to an invitation by t...,en,en,Building an ark creating a vessel for the educ...,This paper is a response to an invitation by t...,NaN,NaN,This paper is a response to an invitation by t...,Building an ark creating a vessel for the educ...
2,2,2,943,mbsr,M33155250,M33155250,['The use of online MBSR audio in medical staf...,"[[, "", O, b, j, e, c, t, i, v, e, !, #, !, T, ...","['Liu, Y', 'Luo, S-X', 'Ye, J-L', 'Chen, Y-Z',...",['European review for medical and pharmacologi...,...,The use of online MBSR audio in medical staff ...,ObjectiveThis study aims to survey medical sta...,en,en,The use of online MBSR audio in medical staff ...,ObjectiveThis study aims to survey medical sta...,NaN,NaN,ObjectiveThis study aims to survey medical sta...,The use of online MBSR audio in medical staff ...
3,3,3,943,mbsr,M30137607,M30137607,['Translational genomic research: the role of ...,"[[, "", G, e, n, e, t, i, c, , v, a, r, i, a, ...","['Park, Jong Y', 'Lengacher, Cecile A', 'Reich...",['Translational behavioral medicine'],...,Translational genomic research the role of gen...,Genetic variations of breast cancer survivors ...,en,en,Translational genomic research the role of gen...,Genetic variations of breast cancer survivors ...,NaN,NaN,Genetic variations of breast cancer survivors ...,Translational genomic research the role of gen...
4,4,4,943,mbsr,M32848377,M32848377,['The Effect of Mindfulness-Based Stress Reduc...,"[[, ', B, a, c, k, g, r, o, u, n, d, !, #, !, ...","['Marciniak, Rafał', 'Šumec, Rastislav', 'Vyhn...",['Clinical interventions in aging'],...,The Effect of MindfulnessBased Stress Reductio...,BackgroundMindfulnessbased programs have shown...,en,en,The Effect of MindfulnessBased Stress Reductio...,BackgroundMindfulnessbased programs have shown...,NaN,NaN,BackgroundMindfulnessbased programs have shown...,The Effect of MindfulnessBased Stress Reductio...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,9995,1172,low AND carb,M33499118,M33499118,['The Effect of Antioxidant and Anti-Inflammat...,"[[, "", C, h, r, o, n, i, c, , i, n, f, l, a, ...","['Katsimbri, Pelagia', 'Korakas, Emmanouil', '...","['Antioxidants (Basel, Switzerland)']",...,The Effect of Antioxidant and AntiInflammatory...,Chronic inflammation and increased oxidative s...,en,en,The Effect of Antioxidant and AntiInflammatory...,Chronic inflammation and increased oxidative s...,NaN,NaN,Chronic inflammation and increased oxidative s...,The Effect of Antioxidant and AntiInflammatory...
9996,9996,9996,1172,low AND carb,M32713393,M32713393,['Participant characteristics and self-reporte...,"[[, "", O, b, j, e, c, t, i, v, e, !, #, !, T, ...","['Karlsen, Micaela C', 'Lichtenstein, Alice H'...",['Public health nutrition'],...,Participant characteristics and selfreported w...,ObjectiveTo describe characteristics of selfid...,en,en,Participant characteristics and selfreported w...,ObjectiveTo describe characteristics of self

## dataframe convert data type into str

In [46]:
df['Preprocessed_Abstract'] = df['Preprocessed_Abstract'].apply(str)
df['Preprocessed_Title']=df['Preprocessed_Title'].apply(str) 

In [47]:
df

,Unnamed: 0,Unnamed: 0.1,qid:,qstr,candidates,DBRECORDID,TITLE,ABSTRACT,AUTHOR,SOURCE,...,TITLE_,ABSTRACT_,TITLE_lan,ABSTRACT_lan,title_en,abstract_en,TITLE_lan code(memo),ABSTRACT_lan code(memo),Preprocessed_Abstract,Preprocessed_Title
0,0,0,943,mbsr,M30928765,M30928765,['The beauty we love: MBSR teacher education g...,['The paper begins with a brief overview of th...,"['Santorelli, Saki F']",['Current opinion in psychology'],...,The beauty we love MBSR teacher education goin...,The paper begins with a brief overview of the ...,en,en,The beauty we love MBSR teacher education goin...,The paper begins with a brief overview of the ...,NaN,NaN,The paper begins with a brief overview of the ...,The beauty we love MBSR teacher education goin...
1,1,1,943,mbsr,M30928764,M30928764,['Building an ark: creating a vessel for the e...,['This paper is a response to an invitation by...,"['Santorelli, Saki F']",['Current opinion in psychology'],...,Building an ark creating a vessel for the educ...,This paper is a response to an invitation by t...,en,en,Building an ark creating a vessel for the educ...,This paper is a response to an invitation by t...,NaN,NaN,This paper is a response to an invitation by t...,Building an ark creating a vessel for the educ...
2,2,2,943,mbsr,M33155250,M33155250,['The use of online MBSR audio in medical staf...,"[""Objective!#!This study aims to survey medica...","['Liu, Y', 'Luo, S-X', 'Ye, J-L', 'Chen, Y-Z',...",['European review for medical and pharmacologi...,...,The use of online MBSR audio in medical staff ...,ObjectiveThis study aims to survey medical sta...,en,en,The use of online MBSR audio in medical staff ...,ObjectiveThis study aims to survey medical sta...,NaN,NaN,ObjectiveThis study aims to survey medical sta...,The use of online MBSR audio in medical staff ...
3,3,3,943,mbsr,M30137607,M30137607,['Translational genomic research: the role of ...,"[""Genetic variations of breast cancer survivor...","['Park, Jong Y', 'Lengacher, Cecile A', 'Reich...",['Translational behavioral medicine'],...,Translational genomic research the role of gen...,Genetic variations of breast cancer survivors ...,en,en,Translational genomic research the role of gen...,Genetic variations of breast cancer survivors ...,NaN,NaN,Genetic variations of breast cancer survivors ...,Translational genomic research the role of gen...
4,4,4,943,mbsr,M32848377,M32848377,['The Effect of Mindfulness-Based Stress Reduc...,['Background!#!Mindfulness-based programs have...,"['Marciniak, Rafał', 'Šumec, Rastislav', 'Vyhn...",['Clinical interventions in aging'],...,The Effect of MindfulnessBased Stress Reductio...,BackgroundMindfulnessbased programs have shown...,en,en,The Effect of MindfulnessBased Stress Reductio...,BackgroundMindfulnessbased programs have shown...,NaN,NaN,BackgroundMindfulnessbased programs have shown...,The Effect of MindfulnessBased Stress Reductio...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,9995,1172,low AND carb,M33499118,M33499118,['The Effect of Antioxidant and Anti-Inflammat...,"[""Chronic inflammation and increased oxidative...","['Katsimbri, Pelagia', 'Korakas, Emmanouil', '...","['Antioxidants (Basel, Switzerland)']",...,The Effect of Antioxidant and AntiInflammatory...,Chronic inflammation and increased oxidative s...,en,en,The Effect of Antioxidant and AntiInflammatory...,Chronic inflammation and increased oxidative s...,NaN,NaN,Chronic inflammation and increased oxidative s...,The Effect of Antioxidant and AntiInflammatory...
9996,9996,9996,1172,low AND carb,M32713393,M32713393,['Participant characteristics and self-reporte...,"[""Objective!#!To describe characteristics of s...","['Karlsen, Micaela C', 'Lichtenstein, Alice H'...",['Public health nutrition'],...,Participant characteristics and selfreported w...,ObjectiveTo describe characteristics of selfid...,en,en,Participant characteristics and selfreported w...,ObjectiveTo describe characteristics of self

In [48]:
def lower_text(text):
    text=text.lower()
    return text 

In [49]:

df['Preprocessed_Abstract']=df['Preprocessed_Abstract'].apply(lower_text)
df['Preprocessed_Title']=df['Preprocessed_Title'].apply(lower_text)

In [50]:
df

,Unnamed: 0,Unnamed: 0.1,qid:,qstr,candidates,DBRECORDID,TITLE,ABSTRACT,AUTHOR,SOURCE,...,TITLE_,ABSTRACT_,TITLE_lan,ABSTRACT_lan,title_en,abstract_en,TITLE_lan code(memo),ABSTRACT_lan code(memo),Preprocessed_Abstract,Preprocessed_Title
0,0,0,943,mbsr,M30928765,M30928765,['The beauty we love: MBSR teacher education g...,['The paper begins with a brief overview of th...,"['Santorelli, Saki F']",['Current opinion in psychology'],...,The beauty we love MBSR teacher education goin...,The paper begins with a brief overview of the ...,en,en,The beauty we love MBSR teacher education goin...,The paper begins with a brief overview of the ...,NaN,NaN,the paper begins with a brief overview of the ...,the beauty we love mbsr teacher education goin...
1,1,1,943,mbsr,M30928764,M30928764,['Building an ark: creating a vessel for the e...,['This paper is a response to an invitation by...,"['Santorelli, Saki F']",['Current opinion in psychology'],...,Building an ark creating a vessel for the educ...,This paper is a response to an invitation by t...,en,en,Building an ark creating a vessel for the educ...,This paper is a response to an invitation by t...,NaN,NaN,this paper is a response to an invitation by t...,building an ark creating a vessel for the educ...
2,2,2,943,mbsr,M33155250,M33155250,['The use of online MBSR audio in medical staf...,"[""Objective!#!This study aims to survey medica...","['Liu, Y', 'Luo, S-X', 'Ye, J-L', 'Chen, Y-Z',...",['European review for medical and pharmacologi...,...,The use of online MBSR audio in medical staff ...,ObjectiveThis study aims to survey medical sta...,en,en,The use of online MBSR audio in medical staff ...,ObjectiveThis study aims to survey medical sta...,NaN,NaN,objectivethis study aims to survey medical sta...,the use of online mbsr audio in medical staff ...
3,3,3,943,mbsr,M30137607,M30137607,['Translational genomic research: the role of ...,"[""Genetic variations of breast cancer survivor...","['Park, Jong Y', 'Lengacher, Cecile A', 'Reich...",['Translational behavioral medicine'],...,Translational genomic research the role of gen...,Genetic variations of breast cancer survivors ...,en,en,Translational genomic research the role of gen...,Genetic variations of breast cancer survivors ...,NaN,NaN,genetic variations of breast cancer survivors ...,translational genomic research the role of gen...
4,4,4,943,mbsr,M32848377,M32848377,['The Effect of Mindfulness-Based Stress Reduc...,['Background!#!Mindfulness-based programs have...,"['Marciniak, Rafał', 'Šumec, Rastislav', 'Vyhn...",['Clinical interventions in aging'],...,The Effect of MindfulnessBased Stress Reductio...,BackgroundMindfulnessbased programs have shown...,en,en,The Effect of MindfulnessBased Stress Reductio...,BackgroundMindfulnessbased programs have shown...,NaN,NaN,backgroundmindfulnessbased programs have shown...,the effect of mindfulnessbased stress reductio...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,9995,1172,low AND carb,M33499118,M33499118,['The Effect of Antioxidant and Anti-Inflammat...,"[""Chronic inflammation and increased oxidative...","['Katsimbri, Pelagia', 'Korakas, Emmanouil', '...","['Antioxidants (Basel, Switzerland)']",...,The Effect of Antioxidant and AntiInflammatory...,Chronic inflammation and increased oxidative s...,en,en,The Effect of Antioxidant and AntiInflammatory...,Chronic inflammation and increased oxidative s...,NaN,NaN,chronic inflammation and increased oxidative s...,the effect of antioxidant and antiinflammatory...
9996,9996,9996,1172,low AND carb,M32713393,M32713393,['Participant characteristics and self-reporte...,"[""Objective!#!To describe characteristics of s...","['Karlsen, Micaela C', 'Lichtenstein, Alice H'...",['Public health nutrition'],...,Participant characteristics and selfreported w...,ObjectiveTo describe characteristics of selfid...,en,en,Participant characteristics and selfreported w...,ObjectiveTo describe characteristics of self

In [51]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text
        

In [52]:

df['Preprocessed_Abstract']=df['Preprocessed_Abstract'].apply(remove_punctuations)
df['Preprocessed_Title']=df['Preprocessed_Title'].apply(remove_punctuations)


In [53]:
def remove_AND_OR(text):
    text=text.replace("and", '')
    text=text.replace("or", '')
    return text
        


In [54]:

df['Preprocessed_Abstract']=df['Preprocessed_Abstract'].apply(remove_AND_OR)
df['Preprocessed_Title']=df['Preprocessed_Title'].apply(remove_AND_OR)


In [55]:
df

,Unnamed: 0,Unnamed: 0.1,qid:,qstr,candidates,DBRECORDID,TITLE,ABSTRACT,AUTHOR,SOURCE,...,TITLE_,ABSTRACT_,TITLE_lan,ABSTRACT_lan,title_en,abstract_en,TITLE_lan code(memo),ABSTRACT_lan code(memo),Preprocessed_Abstract,Preprocessed_Title
0,0,0,943,mbsr,M30928765,M30928765,['The beauty we love: MBSR teacher education g...,['The paper begins with a brief overview of th...,"['Santorelli, Saki F']",['Current opinion in psychology'],...,The beauty we love MBSR teacher education goin...,The paper begins with a brief overview of the ...,en,en,The beauty we love MBSR teacher education goin...,The paper begins with a brief overview of the ...,NaN,NaN,the paper begins with a brief overview of the ...,the beauty we love mbsr teacher education goin...
1,1,1,943,mbsr,M30928764,M30928764,['Building an ark: creating a vessel for the e...,['This paper is a response to an invitation by...,"['Santorelli, Saki F']",['Current opinion in psychology'],...,Building an ark creating a vessel for the educ...,This paper is a response to an invitation by t...,en,en,Building an ark creating a vessel for the educ...,This paper is a response to an invitation by t...,NaN,NaN,this paper is a response to an invitation by t...,building an ark creating a vessel f the educat...
2,2,2,943,mbsr,M33155250,M33155250,['The use of online MBSR audio in medical staf...,"[""Objective!#!This study aims to survey medica...","['Liu, Y', 'Luo, S-X', 'Ye, J-L', 'Chen, Y-Z',...",['European review for medical and pharmacologi...,...,The use of online MBSR audio in medical staff ...,ObjectiveThis study aims to survey medical sta...,en,en,The use of online MBSR audio in medical staff ...,ObjectiveThis study aims to survey medical sta...,NaN,NaN,objectivethis study aims to survey medical sta...,the use of online mbsr audio in medical staff ...
3,3,3,943,mbsr,M30137607,M30137607,['Translational genomic research: the role of ...,"[""Genetic variations of breast cancer survivor...","['Park, Jong Y', 'Lengacher, Cecile A', 'Reich...",['Translational behavioral medicine'],...,Translational genomic research the role of gen...,Genetic variations of breast cancer survivors ...,en,en,Translational genomic research the role of gen...,Genetic variations of breast cancer survivors ...,NaN,NaN,genetic variations of breast cancer survivs bc...,translational genomic research the role of gen...
4,4,4,943,mbsr,M32848377,M32848377,['The Effect of Mindfulness-Based Stress Reduc...,['Background!#!Mindfulness-based programs have...,"['Marciniak, Rafał', 'Šumec, Rastislav', 'Vyhn...",['Clinical interventions in aging'],...,The Effect of MindfulnessBased Stress Reductio...,BackgroundMindfulnessbased programs have shown...,en,en,The Effect of MindfulnessBased Stress Reductio...,BackgroundMindfulnessbased programs have shown...,NaN,NaN,backgroundmindfulnessbased programs have shown...,the effect of mindfulnessbased stress reductio...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,9995,1172,low AND carb,M33499118,M33499118,['The Effect of Antioxidant and Anti-Inflammat...,"[""Chronic inflammation and increased oxidative...","['Katsimbri, Pelagia', 'Korakas, Emmanouil', '...","['Antioxidants (Basel, Switzerland)']",...,The Effect of Antioxidant and AntiInflammatory...,Chronic inflammation and increased oxidative s...,en,en,The Effect of Antioxidant and AntiInflammatory...,Chronic inflammation and increased oxidative s...,NaN,NaN,chronic inflammation increased oxidative stre...,the effect of antioxidant antiinflammaty capa...
9996,9996,9996,1172,low AND carb,M32713393,M32713393,['Participant characteristics and self-reporte...,"[""Objective!#!To describe characteristics of s...","['Karlsen, Micaela C', 'Lichtenstein, Alice H'...",['Public health nutrition'],...,Participant characteristics and selfreported w...,ObjectiveTo describe characteristics of selfid...,en,en,Participant characteristics and selfreported w...,ObjectiveTo describe characteristics of selfid

In [56]:
def double_space(text):
    for i in range(0, len(text)):
        text= text.replace(' +', ' ')
    return text

In [57]:

df['Preprocessed_Abstract']=df['Preprocessed_Abstract'].apply(double_space)
df['Preprocessed_Title']=df['Preprocessed_Title'].apply(double_space)


In [58]:
def remove_special(text):
    ls=[]
    for k in text.split("\n"):
        ls.append(re.sub(r"[^a-zA-Z0-9]+", ' ', k))
    text= " ".join(ls)
    return text


In [59]:

df['Preprocessed_Abstract']=df['Preprocessed_Abstract'].apply(remove_special)
df['Preprocessed_Title']=df['Preprocessed_Title'].apply(remove_special)

In [60]:

df['Preprocessed_Abstract']=df['Preprocessed_Abstract'].apply(nltk.word_tokenize)
df['Preprocessed_Title']=df['Preprocessed_Title'].apply(nltk.word_tokenize)


In [31]:
df

,Unnamed: 0,Unnamed: 0.1,qid:,qstr,candidates,DBRECORDID,TITLE,ABSTRACT,AUTHOR,SOURCE,...,TITLE_,ABSTRACT_,TITLE_lan,ABSTRACT_lan,title_en,abstract_en,TITLE_lan code(memo),ABSTRACT_lan code(memo),Preprocessed_Abstract,Preprocessed_Title
0,0,0,943,mbsr,M30928765,M30928765,['The beauty we love: MBSR teacher education g...,['The paper begins with a brief overview of th...,"['Santorelli, Saki F']",['Current opinion in psychology'],...,The beauty we love MBSR teacher education goin...,The paper begins with a brief overview of the ...,en,en,The beauty we love MBSR teacher education goin...,The paper begins with a brief overview of the ...,NaN,NaN,"[the, paper, begins, with, a, brief, overview,...","[the, beauty, we, love, mbsr, teacher, educati..."
1,1,1,943,mbsr,M30928764,M30928764,['Building an ark: creating a vessel for the e...,['This paper is a response to an invitation by...,"['Santorelli, Saki F']",['Current opinion in psychology'],...,Building an ark creating a vessel for the educ...,This paper is a response to an invitation by t...,en,en,Building an ark creating a vessel for the educ...,This paper is a response to an invitation by t...,NaN,NaN,"[this, paper, is, a, response, to, an, invitat...","[building, an, ark, creating, a, vessel, f, th..."
2,2,2,943,mbsr,M33155250,M33155250,['The use of online MBSR audio in medical staf...,"[""Objective!#!This study aims to survey medica...","['Liu, Y', 'Luo, S-X', 'Ye, J-L', 'Chen, Y-Z',...",['European review for medical and pharmacologi...,...,The use of online MBSR audio in medical staff ...,ObjectiveThis study aims to survey medical sta...,en,en,The use of online MBSR audio in medical staff ...,ObjectiveThis study aims to survey medical sta...,NaN,NaN,"[objectivethis, study, aims, to, survey, medic...","[the, use, of, online, mbsr, audio, in, medica..."
3,3,3,943,mbsr,M30137607,M30137607,['Translational genomic research: the role of ...,"[""Genetic variations of breast cancer survivor...","['Park, Jong Y', 'Lengacher, Cecile A', 'Reich...",['Translational behavioral medicine'],...,Translational genomic research the role of gen...,Genetic variations of breast cancer survivors ...,en,en,Translational genomic research the role of gen...,Genetic variations of breast cancer survivors ...,NaN,NaN,"[genetic, variations, of, breast, cancer, surv...","[translational, genomic, research, the, role, ..."
4,4,4,943,mbsr,M32848377,M32848377,['The Effect of Mindfulness-Based Stress Reduc...,['Background!#!Mindfulness-based programs have...,"['Marciniak, Rafał', 'Šumec, Rastislav', 'Vyhn...",['Clinical interventions in aging'],...,The Effect of MindfulnessBased Stress Reductio...,BackgroundMindfulnessbased programs have shown...,en,en,The Effect of MindfulnessBased Stress Reductio...,BackgroundMindfulnessbased programs have shown...,NaN,NaN,"[backgroundmindfulnessbased, programs, have, s...","[the, effect, of, mindfulnessbased, stress, re..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,9995,1172,low AND carb,M33499118,M33499118,['The Effect of Antioxidant and Anti-Inflammat...,"[""Chronic inflammation and increased oxidative...","['Katsimbri, Pelagia', 'Korakas, Emmanouil', '...","['Antioxidants (Basel, Switzerland)']",...,The Effect of Antioxidant and AntiInflammatory...,Chronic inflammation and increased oxidative s...,en,en,The Effect of Antioxidant and AntiInflammatory...,Chronic inflammation and increased oxidative s...,NaN,NaN,"[chronic, inflammation, increased, oxidative, ...","[the, effect, of, antioxidant, antiinflammaty,..."
9996,9996,9996,1172,low AND carb,M32713393,M32713393,['Participant characteristics and self-reporte...,"[""Objective!#!To describe characteristics of s...","['Karlsen, Micaela C', 'Lichtenstein, Alice H'...",['Public health nutrition'],...,Participant characteristics and selfreported w...,ObjectiveTo describe characteristics of selfid...,en,en,Participant characteristics and selfreported w...,ObjectiveTo describe

In [32]:
def Stopwords(text):
    stop_words = stopwords.words('english')
    word_tokens = [t for t in text if not (t in stop_words)]
    return word_tokens

In [33]:

df['Preprocessed_Abstract']=df['Preprocessed_Abstract'].apply(Stopwords)
df['Preprocessed_Title']=df['Preprocessed_Title'].apply(Stopwords)


In [34]:
df

,Unnamed: 0,Unnamed: 0.1,qid:,qstr,candidates,DBRECORDID,TITLE,ABSTRACT,AUTHOR,SOURCE,...,TITLE_,ABSTRACT_,TITLE_lan,ABSTRACT_lan,title_en,abstract_en,TITLE_lan code(memo),ABSTRACT_lan code(memo),Preprocessed_Abstract,Preprocessed_Title
0,0,0,943,mbsr,M30928765,M30928765,['The beauty we love: MBSR teacher education g...,['The paper begins with a brief overview of th...,"['Santorelli, Saki F']",['Current opinion in psychology'],...,The beauty we love MBSR teacher education goin...,The paper begins with a brief overview of the ...,en,en,The beauty we love MBSR teacher education goin...,The paper begins with a brief overview of the ...,NaN,NaN,"[paper, begins, brief, overview, current, fiel...","[beauty, love, mbsr, teacher, education, going..."
1,1,1,943,mbsr,M30928764,M30928764,['Building an ark: creating a vessel for the e...,['This paper is a response to an invitation by...,"['Santorelli, Saki F']",['Current opinion in psychology'],...,Building an ark creating a vessel for the educ...,This paper is a response to an invitation by t...,en,en,Building an ark creating a vessel for the educ...,This paper is a response to an invitation by t...,NaN,NaN,"[paper, response, invitation, edits, special, ...","[building, ark, creating, vessel, f, education..."
2,2,2,943,mbsr,M33155250,M33155250,['The use of online MBSR audio in medical staf...,"[""Objective!#!This study aims to survey medica...","['Liu, Y', 'Luo, S-X', 'Ye, J-L', 'Chen, Y-Z',...",['European review for medical and pharmacologi...,...,The use of online MBSR audio in medical staff ...,ObjectiveThis study aims to survey medical sta...,en,en,The use of online MBSR audio in medical staff ...,ObjectiveThis study aims to survey medical sta...,NaN,NaN,"[objectivethis, study, aims, survey, medical, ...","[use, online, mbsr, audio, medical, staff, cov..."
3,3,3,943,mbsr,M30137607,M30137607,['Translational genomic research: the role of ...,"[""Genetic variations of breast cancer survivor...","['Park, Jong Y', 'Lengacher, Cecile A', 'Reich...",['Translational behavioral medicine'],...,Translational genomic research the role of gen...,Genetic variations of breast cancer survivors ...,en,en,Translational genomic research the role of gen...,Genetic variations of breast cancer survivors ...,NaN,NaN,"[genetic, variations, breast, cancer, survivs,...","[translational, genomic, research, role, genet..."
4,4,4,943,mbsr,M32848377,M32848377,['The Effect of Mindfulness-Based Stress Reduc...,['Background!#!Mindfulness-based programs have...,"['Marciniak, Rafał', 'Šumec, Rastislav', 'Vyhn...",['Clinical interventions in aging'],...,The Effect of MindfulnessBased Stress Reductio...,BackgroundMindfulnessbased programs have shown...,en,en,The Effect of MindfulnessBased Stress Reductio...,BackgroundMindfulnessbased programs have shown...,NaN,NaN,"[backgroundmindfulnessbased, programs, shown, ...","[effect, mindfulnessbased, stress, reduction, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,9995,1172,low AND carb,M33499118,M33499118,['The Effect of Antioxidant and Anti-Inflammat...,"[""Chronic inflammation and increased oxidative...","['Katsimbri, Pelagia', 'Korakas, Emmanouil', '...","['Antioxidants (Basel, Switzerland)']",...,The Effect of Antioxidant and AntiInflammatory...,Chronic inflammation and increased oxidative s...,en,en,The Effect of Antioxidant and AntiInflammatory...,Chronic inflammation and increased oxidative s...,NaN,NaN,"[chronic, inflammation, increased, oxidative, ...","[effect, antioxidant, antiinflammaty, capacity..."
9996,9996,9996,1172,low AND carb,M32713393,M32713393,['Participant characteristics and self-reporte...,"[""Objective!#!To describe characteristics of s...","['Karlsen, Micaela C', 'Lichtenstein, Alice H'...",['Public health nutrition'],...,Participant characteristics and selfreported w...,ObjectiveTo describe characteristics of selfid...,en,en,Participant characteristics and selfreported w...,ObjectiveTo describe

## Stemming

In [61]:
def stemming(text):
    ps = PorterStemmer()
    stems=[]
    for i in text:
        stems.append(ps.stem(i))
    return stems

In [62]:

df['Preprocessed_Abstract']=df['Preprocessed_Abstract'].apply(stemming)
df['Preprocessed_Title']=df['Preprocessed_Title'].apply(stemming)


In [63]:
df

,Unnamed: 0,Unnamed: 0.1,qid:,qstr,candidates,DBRECORDID,TITLE,ABSTRACT,AUTHOR,SOURCE,...,TITLE_,ABSTRACT_,TITLE_lan,ABSTRACT_lan,title_en,abstract_en,TITLE_lan code(memo),ABSTRACT_lan code(memo),Preprocessed_Abstract,Preprocessed_Title
0,0,0,943,mbsr,M30928765,M30928765,['The beauty we love: MBSR teacher education g...,['The paper begins with a brief overview of th...,"['Santorelli, Saki F']",['Current opinion in psychology'],...,The beauty we love MBSR teacher education goin...,The paper begins with a brief overview of the ...,en,en,The beauty we love MBSR teacher education goin...,The paper begins with a brief overview of the ...,NaN,NaN,"[the, paper, begin, with, a, brief, overview, ...","[the, beauti, we, love, mbsr, teacher, educ, g..."
1,1,1,943,mbsr,M30928764,M30928764,['Building an ark: creating a vessel for the e...,['This paper is a response to an invitation by...,"['Santorelli, Saki F']",['Current opinion in psychology'],...,Building an ark creating a vessel for the educ...,This paper is a response to an invitation by t...,en,en,Building an ark creating a vessel for the educ...,This paper is a response to an invitation by t...,NaN,NaN,"[thi, paper, is, a, respons, to, an, invit, by...","[build, an, ark, creat, a, vessel, f, the, edu..."
2,2,2,943,mbsr,M33155250,M33155250,['The use of online MBSR audio in medical staf...,"[""Objective!#!This study aims to survey medica...","['Liu, Y', 'Luo, S-X', 'Ye, J-L', 'Chen, Y-Z',...",['European review for medical and pharmacologi...,...,The use of online MBSR audio in medical staff ...,ObjectiveThis study aims to survey medical sta...,en,en,The use of online MBSR audio in medical staff ...,ObjectiveThis study aims to survey medical sta...,NaN,NaN,"[objectivethi, studi, aim, to, survey, medic, ...","[the, use, of, onlin, mbsr, audio, in, medic, ..."
3,3,3,943,mbsr,M30137607,M30137607,['Translational genomic research: the role of ...,"[""Genetic variations of breast cancer survivor...","['Park, Jong Y', 'Lengacher, Cecile A', 'Reich...",['Translational behavioral medicine'],...,Translational genomic research the role of gen...,Genetic variations of breast cancer survivors ...,en,en,Translational genomic research the role of gen...,Genetic variations of breast cancer survivors ...,NaN,NaN,"[genet, variat, of, breast, cancer, surviv, bc...","[translat, genom, research, the, role, of, gen..."
4,4,4,943,mbsr,M32848377,M32848377,['The Effect of Mindfulness-Based Stress Reduc...,['Background!#!Mindfulness-based programs have...,"['Marciniak, Rafał', 'Šumec, Rastislav', 'Vyhn...",['Clinical interventions in aging'],...,The Effect of MindfulnessBased Stress Reductio...,BackgroundMindfulnessbased programs have shown...,en,en,The Effect of MindfulnessBased Stress Reductio...,BackgroundMindfulnessbased programs have shown...,NaN,NaN,"[backgroundmindfulnessbas, program, have, show...","[the, effect, of, mindfulnessbas, stress, redu..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,9995,1172,low AND carb,M33499118,M33499118,['The Effect of Antioxidant and Anti-Inflammat...,"[""Chronic inflammation and increased oxidative...","['Katsimbri, Pelagia', 'Korakas, Emmanouil', '...","['Antioxidants (Basel, Switzerland)']",...,The Effect of Antioxidant and AntiInflammatory...,Chronic inflammation and increased oxidative s...,en,en,The Effect of Antioxidant and AntiInflammatory...,Chronic inflammation and increased oxidative s...,NaN,NaN,"[chronic, inflamm, increas, oxid, stress, are,...","[the, effect, of, antioxid, antiinflammati, ca..."
9996,9996,9996,1172,low AND carb,M32713393,M32713393,['Participant characteristics and self-reporte...,"[""Objective!#!To describe characteristics of s...","['Karlsen, Micaela C', 'Lichtenstein, Alice H'...",['Public health nutrition'],...,Participant characteristics and selfreported w...,ObjectiveTo describe characteristics of selfid...,en,en,Participant characteristics and selfreported w...,ObjectiveTo describe

## Change the position

In [64]:
pop1 = df.pop('Preprocessed_Abstract')
pop2 =df.pop('Preprocessed_Title')
df.insert(1, 'Preprocessed_Abstract',pop1)
df.insert(2, 'Preprocessed_Title',pop2)

In [65]:
df

,Unnamed: 0,Preprocessed_Abstract,Preprocessed_Title,Unnamed: 0.1,qid:,qstr,candidates,DBRECORDID,TITLE,ABSTRACT,...,PUBLPLACE,INSTITUTION,TITLE_,ABSTRACT_,TITLE_lan,ABSTRACT_lan,title_en,abstract_en,TITLE_lan code(memo),ABSTRACT_lan code(memo)
0,0,"[the, paper, begin, with, a, brief, overview, ...","[the, beauti, we, love, mbsr, teacher, educ, g...",0,943,mbsr,M30928765,M30928765,['The beauty we love: MBSR teacher education g...,['The paper begins with a brief overview of th...,...,NaN,NaN,The beauty we love MBSR teacher education goin...,The paper begins with a brief overview of the ...,en,en,The beauty we love MBSR teacher education goin...,The paper begins with a brief overview of the ...,NaN,NaN
1,1,"[thi, paper, is, a, respons, to, an, invit, by...","[build, an, ark, creat, a, vessel, f, the, edu...",1,943,mbsr,M30928764,M30928764,['Building an ark: creating a vessel for the e...,['This paper is a response to an invitation by...,...,NaN,NaN,Building an ark creating a vessel for the educ...,This paper is a response to an invitation by t...,en,en,Building an ark creating a vessel for the educ...,This paper is a response to an invitation by t...,NaN,NaN
2,2,"[objectivethi, studi, aim, to, survey, medic, ...","[the, use, of, onlin, mbsr, audio, in, medic, ...",2,943,mbsr,M33155250,M33155250,['The use of online MBSR audio in medical staf...,"[""Objective!#!This study aims to survey medica...",...,NaN,NaN,The use of online MBSR audio in medical staff ...,ObjectiveThis study aims to survey medical sta...,en,en,The use of online MBSR audio in medical staff ...,ObjectiveThis study aims to survey medical sta...,NaN,NaN
3,3,"[genet, variat, of, breast, cancer, surviv, bc...","[translat, genom, research, the, role, of, gen...",3,943,mbsr,M30137607,M30137607,['Translational genomic research: the role of ...,"[""Genetic variations of breast cancer survivor...",...,NaN,NaN,Translational genomic research the role of gen...,Genetic variations of breast cancer survivors ...,en,en,Translational genomic research the role of gen...,Genetic variations of breast cancer survivors ...,NaN,NaN
4,4,"[backgroundmindfulnessbas, program, have, show...","[the, effect, of, mindfulnessbas, stress, redu...",4,943,mbsr,M32848377,M32848377,['The Effect of Mindfulness-Based Stress Reduc...,['Background!#!Mindfulness-based programs have...,...,NaN,NaN,The Effect of MindfulnessBased Stress Reductio...,BackgroundMindfulnessbased programs have shown...,en,en,The Effect of MindfulnessBased Stress Reductio...,BackgroundMindfulnessbased programs have shown...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,"[chronic, inflamm, increas, oxid, stress, are,...","[the, effect, of, antioxid, antiinflammati, ca...",9995,1172,low AND carb,M33499118,M33499118,['The Effect of Antioxidant and Anti-Inflammat...,"[""Chronic inflammation and increased oxidative...",...,NaN,NaN,The Effect of Antioxidant and AntiInflammatory...,Chronic inflammation and increased oxidative s...,en,en,The Effect of Antioxidant and AntiInflammatory...,Chronic inflammation and increased oxidative s...,NaN,NaN
9996,9996,"[objectiveto, describ, characterist, of, selfi...","[particip, characterist, selfrept, weight, sta...",9996,1172,low AND carb,M32713393,M32713393,['Participant characteristics and self-reporte...,"[""Objective!#!To describe characteristics of s...",...,NaN,NaN,Participant characteristics and selfreported w...,ObjectiveTo describe characteristics of selfid...,en,en,Participant characteristics and selfreported w...,ObjectiveTo describe characteristics of selfid...,NaN,NaN
9997,9997,"[backgroundreplac, carbohydr, with, protein, a...","[effect, of, glycem, index, carbohydr, intak, ...",9997,1172,low AND carb,M27391484,M27391484,['Effect of glycemic index and carbohydrate in...,['Background!#!Replacing carbohydrate with pro...,...,NaN,NaN,Effect of glycemic index and carbohydrate inta...,BackgroundReplacing carbohydrate with protein

## Delete Column

In [61]:
del df['ABSTRACT']

In [68]:
df

,Unnamed: 0,Preprocessed_Abstract,Preprocessed_Title,Unnamed: 0.1,qid:,qstr,candidates,DBRECORDID,TITLE,AUTHOR,...,PUBLPLACE,INSTITUTION,TITLE_,ABSTRACT_,TITLE_lan,ABSTRACT_lan,title_en,abstract_en,TITLE_lan code(memo),ABSTRACT_lan code(memo)
0,0,"[paper, begins, brief, overview, current, fiel...","[beauty, love, mbsr, teacher, education, going...",0,943,mbsr,M30928765,M30928765,['The beauty we love: MBSR teacher education g...,"['Santorelli, Saki F']",...,NaN,NaN,The beauty we love MBSR teacher education goin...,The paper begins with a brief overview of the ...,en,en,The beauty we love MBSR teacher education goin...,The paper begins with a brief overview of the ...,NaN,NaN
1,1,"[paper, response, invitation, edits, special, ...","[building, ark, creating, vessel, f, education...",1,943,mbsr,M30928764,M30928764,['Building an ark: creating a vessel for the e...,"['Santorelli, Saki F']",...,NaN,NaN,Building an ark creating a vessel for the educ...,This paper is a response to an invitation by t...,en,en,Building an ark creating a vessel for the educ...,This paper is a response to an invitation by t...,NaN,NaN
2,2,"[objectivethis, study, aims, survey, medical, ...","[use, online, mbsr, audio, medical, staff, cov...",2,943,mbsr,M33155250,M33155250,['The use of online MBSR audio in medical staf...,"['Liu, Y', 'Luo, S-X', 'Ye, J-L', 'Chen, Y-Z',...",...,NaN,NaN,The use of online MBSR audio in medical staff ...,ObjectiveThis study aims to survey medical sta...,en,en,The use of online MBSR audio in medical staff ...,ObjectiveThis study aims to survey medical sta...,NaN,NaN
3,3,"[genetic, variations, breast, cancer, survivs,...","[translational, genomic, research, role, genet...",3,943,mbsr,M30137607,M30137607,['Translational genomic research: the role of ...,"['Park, Jong Y', 'Lengacher, Cecile A', 'Reich...",...,NaN,NaN,Translational genomic research the role of gen...,Genetic variations of breast cancer survivors ...,en,en,Translational genomic research the role of gen...,Genetic variations of breast cancer survivors ...,NaN,NaN
4,4,"[backgroundmindfulnessbased, programs, shown, ...","[effect, mindfulnessbased, stress, reduction, ...",4,943,mbsr,M32848377,M32848377,['The Effect of Mindfulness-Based Stress Reduc...,"['Marciniak, Rafał', 'Šumec, Rastislav', 'Vyhn...",...,NaN,NaN,The Effect of MindfulnessBased Stress Reductio...,BackgroundMindfulnessbased programs have shown...,en,en,The Effect of MindfulnessBased Stress Reductio...,BackgroundMindfulnessbased programs have shown...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,"[chronic, inflammation, increased, oxidative, ...","[effect, antioxidant, antiinflammaty, capacity...",9995,1172,low AND carb,M33499118,M33499118,['The Effect of Antioxidant and Anti-Inflammat...,"['Katsimbri, Pelagia', 'Korakas, Emmanouil', '...",...,NaN,NaN,The Effect of Antioxidant and AntiInflammatory...,Chronic inflammation and increased oxidative s...,en,en,The Effect of Antioxidant and AntiInflammatory...,Chronic inflammation and increased oxidative s...,NaN,NaN
9996,9996,"[objectiveto, describe, characteristics, selfi...","[participant, characteristics, selfrepted, wei...",9996,1172,low AND carb,M32713393,M32713393,['Participant characteristics and self-reporte...,"['Karlsen, Micaela C', 'Lichtenstein, Alice H'...",...,NaN,NaN,Participant characteristics and selfreported w...,ObjectiveTo describe characteristics of selfid...,en,en,Participant characteristics and selfreported w...,ObjectiveTo describe characteristics of selfid...,NaN,NaN
9997,9997,"[backgroundreplacing, carbohydrate, protein, a...","[effect, glycemic, index, carbohydrate, intake...",9997,1172,low AND carb,M27391484,M27391484,['Effect of glycemic index and carbohydrate in...,"['Juraschek, Stephen P', 'Chang, Alex R', 'App...",...,NaN,NaN,Effect of glycemic index and carbohydrate inta...,BackgroundReplacing carbohydrate with protein ...,en,en,Effect of glycemic index and carbohydrate

In [66]:
df.to_csv('PPTranslatedAbstract_Title_WithStopWithStem.csv')